## 10-fold Cross-validation with Ridge regression

In [1]:
# import libraries

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

In [2]:
# define constants

folds = 10
regularization_parameter = np.array([0.01, 0.1, 1, 10, 100])

In [3]:
# load training data

# load data from csv file
data_frame = pd.read_csv ('train.csv')
x_values = data_frame[['x1','x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8','x9','x10','x11','x12','x13']]
y_values = data_frame[['y']]

# visualize data
x_values.info()
y_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      506 non-null    float64
 1   x2      506 non-null    float64
 2   x3      506 non-null    float64
 3   x4      506 non-null    float64
 4   x5      506 non-null    float64
 5   x6      506 non-null    float64
 6   x7      506 non-null    float64
 7   x8      506 non-null    float64
 8   x9      506 non-null    float64
 9   x10     506 non-null    float64
 10  x11     506 non-null    float64
 11  x12     506 non-null    float64
 12  x13     506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   y       506 non-null    float64
dtypes: float64(1)
memory usage: 4.1 KB


In [4]:
# 10-fold cross validation with ridge regression
results = []

for i in range(0 , len(regularization_parameter)):
    ridge_ridgr = linear_model.Ridge(alpha = regularization_parameter[i], normalize= False, fit_intercept = False, max_iter=100000)

    cv_results = cross_validate(ridge_ridgr, x_values, y_values, cv = folds, scoring = 'neg_root_mean_squared_error')
    results.append(np.mean((-1)*cv_results['test_score']))

In [5]:
# save to file

np.savetxt("results.csv", results, delimiter=",")